In [ ]:
Q1

In [ ]:
SELECT 
    L_RETURNFLAG, 
    L_LINESTATUS, 
    SUM(L_QUANTITY) AS SUM_QTY, 
    SUM(L_EXTENDEDPRICE) AS SUM_BASE_PRICE, 
    SUM(DISCOUNT_PRICE(L_EXTENDEDPRICE, L_DISCOUNT)) AS SUM_DISC_PRICE, 
    SUM(DISCOUNT_TAXPRICE(L_EXTENDEDPRICE, L_DISCOUNT, L_TAX)) AS SUM_CHARGE, 
    AVG(L_QUANTITY) AS AVG_QTY, 
    AVG(L_EXTENDEDPRICE) AS AVG_PRICE, 
    AVG(L_DISCOUNT) AS AVG_DISC, 
    COUNT(*) AS COUNT_ORDER
FROM 
    LINEITEM
WHERE 
    ISSHIPPEDBEFORE(L_SHIPDATE, -90, '1998-12-01') = 1
GROUP BY 
    L_RETURNFLAG, 
    L_LINESTATUS
ORDER BY 
    L_RETURNFLAG NULLS FIRST, 
    L_LINESTATUS NULLS FIRST;

In [ ]:
 l_returnflag | l_linestatus |   sum_qty   | sum_base_price  | sum_disc_price  |    sum_charge     |       avg_qty       |     avg_price      |        avg_disc        | count_order
--------------+--------------+-------------+-----------------+-----------------+-------------------+---------------------+--------------------+------------------------+-------------
 A            | F            | 37734107.00 |  56586554400.73 |  53758257382.21 |  55909065479.9960 | 25.5220058532573370 | 38273.129734621672 | 0.04998529583839761162 |     1478493
 N            | F            |   991417.00 |   1487504710.38 |   1413082173.74 |   1469649229.0811 | 25.5164719205229835 | 38284.467760848304 | 0.05009342667421629691 |       38854
 N            | O            | 74476040.00 | 111701729697.74 | 106118230774.99 | 110367044358.6731 | 25.5022267695849915 | 38249.117988908270 | 0.04999658605370408037 |     2920374
 R            | F            | 37719753.00 |  56568041380.90 |  53741292921.05 |  55889619365.8084 | 25.5057936126907707 | 38250.854626099657 | 0.05000940583012705647 |     1478870
(4 行记录)

In [ ]:
Q3

In [ ]:
SELECT 
    L_ORDERKEY, 
    SUM(DISCOUNT_PRICE(L_EXTENDEDPRICE, L_DISCOUNT)) AS REVENUE, 
    O_ORDERDATE, 
    O_SHIPPRIORITY
FROM 
    CUSTOMER, 
    ORDERS, 
    LINEITEM
WHERE 
    C_CUSTKEY = O_CUSTKEY 
    AND L_ORDERKEY = O_ORDERKEY 
    AND Q3CONDITIONS(C_MKTSEGMENT, O_ORDERDATE, L_SHIPDATE) = 1
GROUP BY 
    L_ORDERKEY, 
    O_ORDERDATE, 
    O_SHIPPRIORITY
ORDER BY 
    REVENUE DESC NULLS LAST, 
    O_ORDERDATE NULLS FIRST
LIMIT 
    10;

In [ ]:
 l_orderkey |  revenue  | o_orderdate | o_shippriority
------------+-----------+-------------+----------------
    2456423 | 406181.00 | 1995-03-05  |              0
    3459808 | 405838.70 | 1995-03-04  |              0
     492164 | 390324.06 | 1995-02-19  |              0
    1188320 | 384537.93 | 1995-03-09  |              0
    2435712 | 378673.06 | 1995-02-26  |              0
    4878020 | 378376.79 | 1995-03-12  |              0
    5521732 | 375153.93 | 1995-03-13  |              0
    2628192 | 373133.31 | 1995-02-22  |              0
     993600 | 371407.45 | 1995-03-05  |              0
    2300070 | 367371.14 | 1995-03-13  |              0
(10 行记录)

In [ ]:
Q5

In [ ]:
SELECT 
    N_NAME, 
    SUM(DISCOUNT_PRICE(L_EXTENDEDPRICE, L_DISCOUNT)) AS REVENUE
FROM 
    CUSTOMER, 
    ORDERS, 
    LINEITEM, 
    SUPPLIER, 
    NATION, 
    REGION
WHERE 
    C_CUSTKEY = O_CUSTKEY 
    AND L_ORDERKEY = O_ORDERKEY 
    AND L_SUPPKEY = S_SUPPKEY 
    AND C_NATIONKEY = S_NATIONKEY 
    AND S_NATIONKEY = N_NATIONKEY 
    AND N_REGIONKEY = R_REGIONKEY 
    AND Q5CONDITIONS(R_NAME, O_ORDERDATE) = 1
GROUP BY 
    N_NAME
ORDER BY 
    REVENUE DESC NULLS LAST;

In [ ]:
          n_name           |   revenue
---------------------------+-------------
 INDONESIA                 | 55502041.37
 VIETNAM                   | 55295087.24
 CHINA                     | 53724494.43
 INDIA                     | 52035512.19
 JAPAN                     | 45410175.92
(5 行记录)

In [ ]:
Q6

In [ ]:
SELECT 
    SUM(L_EXTENDEDPRICE * L_DISCOUNT) AS REVENUE
FROM 
    LINEITEM
WHERE 
    Q6CONDITIONS(
        L_SHIPDATE::DATE, 
        L_DISCOUNT::DECIMAL(12,2), 
        L_QUANTITY::INT
    ) = 1;

In [ ]:
    revenue
----------------
 123141078.2283
(1 行记录)

In [ ]:
Q7

In [ ]:
SELECT 
    SUPP_NATION, 
    CUST_NATION, 
    L_YEAR, 
    SUM(VOLUME) AS REVENUE
FROM 
    (
        SELECT 
            N1.N_NAME AS SUPP_NATION, 
            N2.N_NAME AS CUST_NATION, 
            TO_CHAR(CAST(L_SHIPDATE AS TIMESTAMP), 'YY') AS L_YEAR, 
            L_EXTENDEDPRICE * (1 - L_DISCOUNT) AS VOLUME
        FROM 
            SUPPLIER, 
            LINEITEM, 
            ORDERS, 
            CUSTOMER, 
            NATION AS N1, 
            NATION AS N2
        WHERE 
            S_SUPPKEY = L_SUPPKEY 
            AND O_ORDERKEY = L_ORDERKEY 
            AND C_CUSTKEY = O_CUSTKEY 
            AND S_NATIONKEY = N1.N_NATIONKEY 
            AND C_NATIONKEY = N2.N_NATIONKEY 
            AND Q7CONDITIONS(N1.N_NAME, N2.N_NAME, L_SHIPDATE) = 1
    ) AS SHIPPING
GROUP BY 
    SUPP_NATION, 
    CUST_NATION, 
    L_YEAR
ORDER BY 
    SUPP_NATION NULLS FIRST, 
    CUST_NATION NULLS FIRST, 
    L_YEAR NULLS FIRST;

In [ ]:
        supp_nation        |        cust_nation        | l_year |    revenue
---------------------------+---------------------------+--------+---------------
 FRANCE                    | GERMANY                   | 95     | 54639732.7336
 FRANCE                    | GERMANY                   | 96     | 54633083.3076
 GERMANY                   | FRANCE                    | 95     | 52531746.6697
 GERMANY                   | FRANCE                    | 96     | 52520549.0224
(4 行记录)

In [ ]:
Q9

In [ ]:
SELECT 
    NATION, 
    O_YEAR, 
    SUM(AMOUNT) AS SUM_PROFIT
FROM 
    (
        SELECT 
            N_NAME AS NATION, 
            TO_CHAR(CAST(O_ORDERDATE AS TIMESTAMP), 'YY') AS O_YEAR, 
            PROFIT_AMOUNT(L_EXTENDEDPRICE, L_DISCOUNT, PS_SUPPLYCOST, L_QUANTITY::INT) AS AMOUNT
        FROM 
            PART, 
            SUPPLIER, 
            LINEITEM, 
            PARTSUPP, 
            ORDERS, 
            NATION
        WHERE 
            S_SUPPKEY = L_SUPPKEY 
            AND PS_SUPPKEY = L_SUPPKEY 
            AND PS_PARTKEY = L_PARTKEY 
            AND P_PARTKEY = L_PARTKEY 
            AND O_ORDERKEY = L_ORDERKEY 
            AND S_NATIONKEY = N_NATIONKEY 
            AND P_NAME LIKE '%green%'
    ) AS PROFIT
GROUP BY 
    NATION, 
    O_YEAR
ORDER BY 
    NATION NULLS FIRST, 
    O_YEAR DESC NULLS LAST
LIMIT 10;

In [ ]:
          nation           | o_year |  sum_profit
---------------------------+--------+---------------
 ALGERIA                   | 98     | 27136900.1803
 ALGERIA                   | 97     | 48611833.4962
 ALGERIA                   | 96     | 48285482.6782
 ALGERIA                   | 95     | 44402273.5999
 ALGERIA                   | 94     | 48694008.0668
 ALGERIA                   | 93     | 46044207.7838
 ALGERIA                   | 92     | 45636849.4881
 ARGENTINA                 | 98     | 28341663.7848
 ARGENTINA                 | 97     | 47143964.1176
 ARGENTINA                 | 96     | 45255278.6021
(10 行记录)

In [ ]:
Q10

In [ ]:
SELECT 
    C_CUSTKEY, 
    C_NAME, 
    SUM(DISCOUNT_PRICE(L_EXTENDEDPRICE, L_DISCOUNT)) AS REVENUE, 
    C_ACCTBAL, 
    N_NAME, 
    C_ADDRESS, 
    C_PHONE, 
    C_COMMENT
FROM 
    CUSTOMER, 
    ORDERS, 
    LINEITEM, 
    NATION
WHERE 
    C_CUSTKEY = O_CUSTKEY 
    AND L_ORDERKEY = O_ORDERKEY 
    AND Q10CONDITIONS(O_ORDERDATE, L_RETURNFLAG) = 1 
    AND C_NATIONKEY = N_NATIONKEY
GROUP BY 
    C_CUSTKEY, 
    C_NAME, 
    C_ACCTBAL, 
    C_PHONE, 
    N_NAME, 
    C_ADDRESS, 
    C_COMMENT
ORDER BY 
    REVENUE DESC NULLS LAST
LIMIT 
    10;

In [ ]:
 c_custkey |       c_name       |  revenue  | c_acctbal |          n_name           |                c_address                 |     c_phone     |                                                  c_comment                                   
-----------+--------------------+-----------+-----------+---------------------------+------------------------------------------+-----------------+-------------------------------------------------------------------------------------------------------------
     57040 | Customer#000057040 | 734235.24 |    632.87 | JAPAN                     | Eioyzjf4pp                               | 22-895-641-3466 | sits. slyly regular requests sleep alongside of the regular inst
    143347 | Customer#000143347 | 721002.68 |   2557.47 | EGYPT                     | 1aReFYv,Kw4                              | 14-742-935-3718 | ggle carefully enticing requests. final deposits use bold, bold pinto beans. ironic, idle re
     60838 | Customer#000060838 | 679127.31 |   2454.77 | BRAZIL                    | 64EaJ5vMAHWJlBOxJklpNc2RJiWE             | 12-913-494-9813 |  need to boost against the slyly regular account
    101998 | Customer#000101998 | 637029.57 |   3790.89 | UNITED KINGDOM            | 01c9CILnNtfOQYmZj                        | 33-593-865-6378 | ress foxes wake slyly after the bold excuses. ironic platelets are furiously carefully bold theodolites
    125341 | Customer#000125341 | 633508.08 |   4983.51 | GERMANY                   | S29ODD6bceU8QSuuEJznkNaK                 | 17-582-695-5962 | arefully even depths. blithely even excuses sleep furiously. foxes use except the dependencies. ca
     25501 | Customer#000025501 | 620269.79 |   7725.04 | ETHIOPIA                  |   W556MXuoiaYCCZamJI,Rn0B4ACUGdkQ8DZ     | 15-874-808-6793 | he pending instructions wake carefully at the pinto beans. regular, final instructions along the slyly fina
    115831 | Customer#000115831 | 596423.88 |   5098.10 | FRANCE                    | rFeBbEEyk dl ne7zV5fDrmiq1oK09wV7pxqCgIc | 16-715-386-3788 | l somas sleep. furiously final deposits wake blithely regular pinto b
     84223 | Customer#000084223 | 594998.03 |    528.65 | UNITED KINGDOM            | nAVZCs6BaWap rrM27N 2qBnzc5WBauxbA       | 33-442-824-8191 |  slyly final deposits haggle regular, pending dependencies. pending escapades wake
     54289 | Customer#000054289 | 585603.38 |   5583.02 | IRAN                      | vXCxoCsU0Bad5JQI ,oobkZ                  | 20-834-292-4707 | ely special foxes are quickly finally ironic p
     39922 | Customer#000039922 | 584878.12 |   7321.11 | GERMANY                   | Zgy4s50l2GKN4pLDPBU8m342gIw6R            | 17-147-757-8036 | y final requests. furiously final foxes cajole blithely special platelets. f
(10 行记录)

In [ ]:
Q11

In [ ]:
SELECT 
    PS_PARTKEY, 
    SUM(PS_SUPPLYCOST * PS_AVAILQTY) AS VALUE
FROM 
    PARTSUPP, 
    SUPPLIER, 
    NATION
WHERE 
    PS_SUPPKEY = S_SUPPKEY 
    AND S_NATIONKEY = N_NATIONKEY 
    AND N_NAME = 'GERMANY'
GROUP BY 
    PS_PARTKEY
HAVING 
    SUM(PS_SUPPLYCOST * PS_AVAILQTY) > TOTAL_VALUE()
ORDER BY 
    VALUE DESC NULLS LAST;

In [ ]:
 ps_partkey |    value
------------+-------------
     129760 | 17538456.86
     166726 | 16503353.92
     191287 | 16474801.97
     161758 | 16101755.54
      34452 | 15983844.72
     139035 | 15907078.34
       9403 | 15451755.62
     154358 | 15212937.88
      38823 | 15064802.86
      85606 | 15053957.15

In [ ]:
Q12

In [ ]:
SELECT 
    L_SHIPMODE, 
    SUM(LINE_COUNT(O_ORDERPRIORITY, 'high')) AS HIGH_LINE_COUNT, 
    SUM(LINE_COUNT(O_ORDERPRIORITY, 'low')) AS LOW_LINE_COUNT
FROM 
    ORDERS, 
    LINEITEM
WHERE 
    O_ORDERKEY = L_ORDERKEY 
    AND Q12CONDITIONS(L_SHIPMODE, L_COMMITDATE, L_RECEIPTDATE, L_SHIPDATE) = 1
GROUP BY 
    L_SHIPMODE
ORDER BY 
    L_SHIPMODE NULLS FIRST;

In [ ]:
 l_shipmode | high_line_count | low_line_count
------------+-----------------+----------------
 MAIL       |           10746 |              0
 SHIP       |           10864 |              0

In [ ]:
Q14

In [ ]:
SELECT 
    100.00 * SUM(PROMO_DISC(P_TYPE, L_EXTENDEDPRICE, L_DISCOUNT)) / SUM(DISCOUNT_PRICE(L_EXTENDEDPRICE, L_DISCOUNT)) AS PROMO_REVENUE
FROM 
    LINEITEM, 
    PART
WHERE 
    L_PARTKEY = P_PARTKEY 
    AND L_SHIPDATE >= DATE '1995-09-01' 
    AND L_SHIPDATE < DATE '1995-09-01' + INTERVAL '1 MONTH';

In [ ]:
    promo_revenue
---------------------
 16.3807786354670190
(1 行记录)

In [ ]:
Q19

In [ ]:
SELECT 
    SUM(DISCOUNT_PRICE(L_EXTENDEDPRICE, L_DISCOUNT)) AS REVENUE
FROM 
    LINEITEM
JOIN 
    PART 
ON 
    L_PARTKEY = P_PARTKEY
WHERE 
    Q19CONDITIONS(P_CONTAINER, L_QUANTITY::INT, P_SIZE, L_SHIPMODE, L_SHIPINSTRUCT, P_BRAND) = 1;

In [ ]:
  revenue
------------
 3083843.03
(1 行记录)

In [ ]:
Q22

In [ ]:
SELECT 
    CNTRYCODE, 
    COUNT(*) AS NUMCUST, 
    SUM(C_ACCTBAL) AS TOTACCTBAL
FROM 
    (
        SELECT 
            SUBSTRING(C_PHONE FROM 1 FOR 2) AS CNTRYCODE, 
            C_ACCTBAL
        FROM 
            CUSTOMER
        WHERE 
            SUBSTRING(C_PHONE FROM 1 FOR 2) IN ('13', '31', '23', '29', '30', '18', '17') 
            AND C_ACCTBAL > AVG_ACTBAL() 
            AND NOT EXISTS (
                SELECT 
                    * 
                FROM 
                    ORDERS 
                WHERE 
                    O_CUSTKEY = C_CUSTKEY
            )
    ) AS CUSTSALE
GROUP BY 
    CNTRYCODE
ORDER BY 
    CNTRYCODE NULLS FIRST;

In [ ]:
 cntrycode | numcust | totacctbal
-----------+---------+------------
 13        |     888 | 6737713.99
 17        |     861 | 6460573.72
 18        |     964 | 7236687.40
 23        |     892 | 6701457.95
 29        |     948 | 7158866.63
 30        |     909 | 6808436.13
 31        |     922 | 6806670.18
(7 行记录)